# CNN/GNN/LSTM/Transformer Model Test

## Import dependencies and data

In [ ]:
# import dependencies
import os
import json
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# define relevant paths
home_path = os.path.dirname(os.getcwd())
data_path = home_path + '\\data\\'
plot_path = home_path + '\\plotting\\plots\\'

In [ ]:
# get merged data
data = pd.read_csv(
    data_path + 'data_merged.csv',
    header=[0, 1],
    index_col=[0, 1, 2, 3]
)

# convert index.date col to datetime
data.index = data.index.set_levels(
    levels=pd.to_datetime(data.index.get_level_values(3).values),
    level=3
)

In [ ]:
# define X vars
x_vars = [
    ('pressure', 'DC_GP101'),
    ('pressure', 'SMHI'),
    ('flow', 'TA01_GP101_default'),
    ('flow', 'FF01_GP101_default'),
    ('temperatures', 'DC_GT401_GM401'),
    ('temperatures', 'TA01_GT10X_GM10X'),
    ('temperatures', 'DC_GT301_damped'),
    ('temperatures', 'DC_GT301_outdoor'),
    ('humidity', 'TA01_GT10X_GM10X'),
    ('humidity', 'DC_GT401_GM401'),
    ('humidity', 'SMHI'),
    ('wind', 'SMHI_direction'),
    ('wind', 'SMHI_speed'),
    ('setpoints', 'DC_GP101_default'),
    ('setpoints', 'TA01_GT10X_GM10X_default')
]

# filter columns to keep only x_vars
data = data[x_vars].copy()

## Describe Df

In [ ]:
# print number of NAs
print('Number of NAs\n')
print(data.isna().sum())
print('\n\n')

# check for NaNs
nadat = data.droplevel(level=0, axis=1)[[
    'TA01_GP101_default', 
    'FF01_GP101_default',
    'DC_GP101_default',
    'TA01_GT10X_GM10X_default'
]]

# delete Nan-data
del nadat

In [ ]:
data['mins'] = data.index.get_level_values(3)
data['mins'] = data.mins.apply(lambda d: d.strftime('%m-%d-%Y %H:%M'))

data = data.groupby(['mins']).agg('mean')


## Process / transform variables 

In [ ]:
# outdoors pressure in kPa-ish
data[('pressure', 'SMHI')] = data.pressure.SMHI.values - data.pressure.SMHI.min()

# remove erroneous setpoints data
data[data.setpoints.TA01_GT10X_GM10X_default == 0.0].TA01_GT10X_GM10X_default = 20.0
data[data.setpoints.DC_GP101_default == 0.0].DC_GP101_default = 4.0

# Use temperature Diff. w.r.t. setpoint
data[('temperatures', 'TA01_GT10X_GM10X_diff')] = \
    data[('temperatures', 'TA01_GT10X_GM10X')] - data[('setpoints', 'TA01_GT10X_GM10X_default')]

# Use rolling avg. f. indoor humidity to filter out 
# effect of humidifier
data[('humidity', 'TA01_GT10X_GM10X')] = data.humidity.TA01_GT10X_GM10X.rolling(
    window=10,
    center=True
).mean().values

# Use rolling avg. DC-GH humidity
#data[('humidity', 'DC_GT401_GM401')] = data.humidity.DC_GT401_GM401.rolling(
#    window=10,
#    center=True,
#).mean().values

# remove NaNs from rolling avgs. (i.e. first 10 observations in data)
data = data[data.humidity.TA01_GT10X_GM10X.notnull()]

In [ ]:
# create variable f. time to account for time-specific effects (e.g. transpiration)

#data[('time', 'minutes')] = data.index.get_level_values(3)
data[('time', 'minutes')] = data.index.get_level_values(0)
data[('time', 'min_sin')] = data.time.minutes.apply(
    lambda d: abs(np.sin((d.hour * 60 + d.minute) * (2 * np.pi / (24 * 60 * 60))))
)
data[('time', 'min_cos')] = data.time.minutes.apply(
    lambda d: abs(np.cos((d.hour * 60 + d.minute) * (2 * np.pi / (24 * 60 * 60))))
)

In [ ]:
# create variable f. wind distribution
wv = data.pop(('wind', 'SMHI_speed'))

# Convert to radians.
wd_rad = data.pop(('wind', 'SMHI_direction'))*np.pi / 180

# Calculate the wind x and y components.
data[('wind', 'Wx')] = wv*np.cos(wd_rad)
data[('wind', 'Wy')] = wv*np.sin(wd_rad)

### Plot pairwise correlation matrix

In [ ]:
corr = data.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 6))
sns.heatmap(corr, mask=mask, vmin=-1.0, vmax=1.0, annot=True)

plt.xticks(rotation=45)
plt.title('Pairwise correlation plot')
plt.show()

# delete correlation data
del corr

## Preparing data

In [ ]:
### SET N_STEPS
n_steps = 30
t_steps = 10

### MASK f. TRAIN/TEST SPLIT
mask = data.index.get_level_values(0) == 2

### GET MODEL VARS
model_vars = [
    ('pressure', 'DC_GP101'),
    ('pressure', 'SMHI'),
    ('flow', 'TA01_GP101_default'),
    ('flow', 'FF01_GP101_default'),
    ('temperatures', 'DC_GT401_GM401'),
    ('temperatures', 'TA01_GT10X_GM10X'),
    ('temperatures', 'TA01_GT10X_GM10X_diff'),
    ('temperatures', 'DC_GT301_damped'),
    ('temperatures', 'DC_GT301_outdoor'),
    ('humidity', 'TA01_GT10X_GM10X'),
    ('humidity', 'DC_GT401_GM401'),
    ('humidity', 'SMHI'),
    ('wind', 'Wx'),
    ('wind', 'Wy'),
    #('setpoints', 'DC_GP101_default'),
    #('setpoints', 'TA01_GT10X_GM10X_default'),
    #('time', 'min_sin'),
    #('time', 'min_cos')
]

In [ ]:
# get train & test data
data_train = data[~mask][model_vars].copy()
data_test  = data[mask][model_vars].copy()

# delete original dataset & filtering mask
del data, mask

# get mean & std f. whitening
mean = np.mean(
    data_train.values, 
    axis=0
)

std  = np.std(
    data_train.values, 
    axis=0
)

### Erroneous seqeunces

Since we have sequential data and missing/removed days, we will have some outlier series for which there is an "erroneous" break. Since e.g. December $16$ is removed from data, there will be sequences for which data starts December $15$ and continues/end in December $17$, and so on. We want to deal with these cases by removing the first number of sequences in a day corresponding to the number of **n_steps**, for when that day is preceded by a *deleted day*.

In [ ]:
def date_flagger(data, n_steps):
    # get flagged dates
    data['day'] = data.index.get_level_values(1).values
    data['date_flag'] = data.day - data.day.shift(periods=n_steps) > 1

    # get positions in data, w.r.t. n_step removed observations at start
    flagged_idx = np.where(data.date_flag.values == 1)
    flagged_idx = flagged_idx[0] - n_steps
    
    del data['day'], data['date_flag']
    
    return flagged_idx

def data_split(data, n_steps, t_steps, mean, std):
    """
    Split and whiten data. Using training mean and s.d. also for test data.
    """
    # get flagged dates, accounting for clipped 
    flags = date_flagger(
        data, 
        n_steps
    )
    
    # get numeric data
    dvals = (data.values - mean) / std
    n = len(data)
    
    # delete dataset
    del data
    
    # get all n_step-sequences from data series
    sequences = np.stack([
        dvals[i:i+n_steps, :] for i in range(n - n_steps)
    ]) 
    
    # get mask f. flags
    mask = [idx not in flags for idx in range(len(sequences))]
    sequences = sequences[mask]
    
    # get targets, delete mask
    targets = dvals[n_steps:][mask]
    del mask
    
    # clip t_steps from sequences and targets
    sequences = sequences[:-t_steps]
    targets = targets[t_steps:]
    
    # return sequences w. channel
    #return sequences[..., np.newaxis]
    return sequences, targets

In [ ]:
# get sequences f. training and test data
X_train, Y_train = data_split(data_train, n_steps=n_steps, t_steps=t_steps, mean=mean, std=std)
X_test, Y_test = data_split(data_test, n_steps=n_steps, t_steps=t_steps, mean=mean, std=std)

# filter Y-data
####Y_train = Y_train[mask]
Y_train = Y_train[:, 6]
Y_test = Y_test[:, 6]

# delete data sets
del data_train, data_test

## Define model

In [ ]:
layerList = [
    layers.Input(shape=(
        X_train.shape[1],
        X_train.shape[2]
    )),
    layers.Conv1D(
        filters=128,
        kernel_size=7, 
        padding="same", 
        strides=2, 
        activation="relu"
    ),
    layers.Dropout(rate=0.5),
    layers.Conv1D(
        filters=64,
        kernel_size=7, 
        padding="same", 
        strides=2, 
        activation="relu"
    ),
    layers.Dropout(rate=0.5),
    #layers.Flatten(),
    layers.LSTM(
        units=32,
        activation='tanh',
        recurrent_activation='sigmoid',
    ),
    #layers.Dense(128, activation='relu'),
    #layers.Dense(256, activation='relu'),
    layers.Dense(1, activation=None)
]

In [ ]:
model = tf.keras.models.Sequential(layers=layerList, name='model')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

In [ ]:
trainHist = model.fit(
    X_train,
    Y_train,
    epochs=20,
    batch_size=128,
    validation_split=0.1,
    #callbacks=[
    #    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    #],
)

In [ ]:
preds = model.predict(X_test)